In [ ]:
#[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# Covid-19 Dashboard

Coronavirus disease 2019 (COVID-19) is a contagious disease caused by the coronavirus 2 that causes severe acute respiratory syndrome (SARS-CoV-2). In December of this year, the first known case was discovered in Wuhan, China. Since then, the disease has spread worldwide, resulting in a pandemic.

One of the most important considerations concerning a novel infectious disease like the new coronavirus is how lethal it is. After months of research, it was discovered that COVID-19 can be fatal to some people. Older adults and people with existing health conditions are more at risk of serious complications, which can be life threatening.

In [ ]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [ ]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Fatalities across England


In [ ]:
with open("genderdeaths.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

In [ ]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
jsondata={}

Since the beginning of COVID-19 in England, vast amounts of data have been collected to provide a clearer image of the virus. Fatalities across England in both men and women have been measured; the data has shown that men are more likely to die from COVID-19 than women. Below is a graph highlighting the number of deaths across England as a nation in both men and women and the total number of deaths.

In [ ]:
def min_age(agerange):
    agerange=agerange.replace('+','')
    start=agerange.split('_')[0]
    return int(start)

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    jsondatadic=jsondata['data'][0] 
    maleDeaths=jsondatadic['maleDeaths']
    femaleDeaths=jsondatadic['femaleDeaths']
    ageranges=[x['age'] for x in maleDeaths] 
    ageranges.sort(key=min_age)
    genderdeath_df=pd.DataFrame(index=ageranges, columns=['femaleDeaths','malesDeaths', 'totalDeaths'])
    # we have no real data to wrangle, so we just generate two random walks.
    for entry in femaleDeaths:
        ageband=entry['age']
        genderdeath_df.loc[ageband, 'femaleDeaths']=entry['value']
    for entry in maleDeaths: # each entry is a dictionary
        ageband=entry['age'] # our index position
        genderdeath_df.loc[ageband, 'maleDeaths']=entry['value']

    genderdeath_df['totalDeaths']=genderdeath_df['maleDeaths']+genderdeath_df['femaleDeaths']

    return genderdeath_df


# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
genderdeath_df=wrangle_data(jsondata) # df is the dataframe for plotting

In [ ]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
    'areaType=nation',
    'areaName=england'
    ]

    structure = {
    "date": "date",
    "femaleDeaths": "femaleDeaths28Days",
    "maleDeaths": "maleDeaths28Days"
    }

    api = Cov19API(filters=filters, structure=structure)

    genderdeaths=api.get_json()
    return {} # return data read from the API

In [ ]:
def death_graph(graphcolumns):
    ncols=len(graphcolumns)
    if ncols>0:
        genderdeath_df.plot(kind='bar', y=list(graphcolumns))
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

deathcols=wdg.SelectMultiple(
    options=['femaleDeaths', 'maleDeaths', 'totalDeaths'], 
    value=['femaleDeaths', 'maleDeaths'], 
    rows=3, 
    description='Sex',
    disabled=False
)
def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=deathcols.value
    deathcols.value=[]
    deathcols.value=current
    


output=wdg.interactive_output(death_graph, {'graphcolumns': deathcols})

display(deathcols, output)

In [ ]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global genderdeath_df
    genderdeath_df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh data', # you may want to change this...
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to update the graph with new data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='sync'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

## Roll out of the vaccine

Since the outbreak, strenuous efforts have been made to produce and roll out a COVID-19 vaccine. As of the end of 2020, a vaccine was approved for use in England and has since been rolled out. Below is a graph detailing the number of first doses, second doses and third/booster doses that have since been administered to the public.

In [ ]:
#with open("vaccination.json", "rt") as INFILE:
    #data=json.load(INFILE)

In [ ]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
#data={}

In [ ]:

#def parse_date(datestring):
    #""" Convert a date string into a pandas datetime object """
    #return pd.to_datetime(datestring, format="%Y-%m-%d")


In [1]:
#def wrangle_data(rawdata):
  # """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
   #Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    #datalist=data['data']
   # dates=[dictionary['date'] for dictionary in datalist ]
   # dates.sort()
   # dates
   # startdate=parse_date(dates[0])
   # enddate=parse_date(dates[-1])
    #index=pd.date_range(startdate, enddate, freq='D')
    #vaccinationdf=pd.DataFrame(index=index, columns=['firstDose', 'secondDose', 'thirdDoseAndBooster'])
    #for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
    #    date=parse_date(entry['date'])
     #   for column in ['firstDose', 'secondDose', 'thirdDoseAndBooster']:
      #      if pd.isna(vaccinationdf.loc[date, column]): 
       #         value= float(entry[column]) if entry[column]!=None else 0.0
        #        vaccinationdf.loc[date, column]=value
            
   #vaccinationdf.fillna(0.0, inplace=True)
   #return vaccinationdf

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
#accinationdf=wrangle_data(jsondata2) # df is the dataframe for plotting

In [2]:
#def access_api():
  #  """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
   # filters = [
    #'areaType=nation',
  #  'areaName=England'
 #   ]


# values here are the names of the PHE metrics
  #  structure = {
  #  "date": "date",
  #  "firstDose": "cumPeopleVaccinatedFirstDoseByPublishDate",
   # "secondDose": "cumPeopleVaccinatedSecondDoseByPublishDate",
  #  "thirdDoseAndBooster": "cumPeopleVaccinatedThirdInjectionByPublishDate"
  #  }
  #  api = Cov19API(filters=filters, structure=structure)

   # vaccination=api.get_json()
  #  return {} # return data read from the API

In [3]:
#def vaccination_graph(ccols, cscale):
 #   if cscale == 'linear':
     #  logscale=False
  #  else:
    #    logscale=True
   # scols=len(ccols)
   # if scols>0:
     #   vaccinationdf[list(ccols)].plot(logy=logscale)
    #    plt.show()
   # else:
     #   print("Click to select data for graph")
     #   print("(CTRL-Click to select more than one category)")

#vaccination=wdg.SelectMultiple(
  #  options=['firstDose', 'secondDose','thirdDoseAndBooster'],
  #  value=['firstDose', 'secondDose','thirdDoseAndBooster'],
  #  rows=3,
   # description='Stats:',
   # disabled=False
#)
#LogorLinear=wdg.ToggleButtons(
  #  options=['log', 'linear'],
  #  description='Scale:',
   # disabled=False,
   # button_style='', # 'success', 'info', 'warning', 'danger' or ''
   # tooltips=['Log Scale', 'Linear Scale'],
#     icons=['check'] * 3
#)
#control=wdg.HBox([LogorLinear, vaccination])
#ctrls=wdg.VBox([LogorLinear, vaccination])

#def refresh_graph_two():
  #  """ We change the value of the widget in order to force a redraw of the graph;
  #  this is useful when the data have been updated. This is a bit of a gimmick; it
   # needs to be customised for one of your widgets. """
    #current=vaccination.value
    #vaccination.value=[]
    #vaccination.value=current
    
#vaccinationgraph=wdg.interactive_output(vaccination_graph, {'ccols': vaccination, 'cscale': LogorLinear})

#form=wdg.HBox([vaccinationgraph, ctrls])
#display(form)


In [4]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
#def api_button_callback1(button):
 #   """ Button callback - it must take the button as its parameter (unused in this case).
 #   Accesses API, wrangles data, updates global variable df used for plotting. """
  #  # Get fresh data from the API. If you have time, include some error handling
  #  # around this call.
   # apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
   # global vaccinationdf
  #  vaccinationdf=wrangle_data(apidata)
  # refresh_graph_two()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
  #  apibutton.icon="check"
    # apibutton.disabled=True

    
#apibutton=wdg.Button(
  #  description='Refresh data', # you may want to change this...
   # disabled=False,
    #button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    #tooltip="Click to update the graph with new data",
    # FontAwesome names without the `fa-` prefix - try "download"
   # icon='sync'
#)

# remember to register your button callback function with the button
#apibutton.on_click(api_button_callback) # the name of your function inside these brackets

#display(apibutton)

# run all cells before clicking on this button

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*